An efficient approach to adapt existing dense retrieval models to new domains & data.
Search results are better as compared to existing model


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Necessary Imports

In [2]:
!pip install -U pinecone-client==2.0.10
!pip install opendatasets
!pip install sentence_transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 159 kB 22.2 MB/s 
     |████████████████████████████████| 269 kB 63.6 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.5 MB/s 
     |████████████████████████████████| 365 kB 59.1 MB/s 
     |████████████████████████████████| 4.7 MB 56.0 MB/s 
     |████████████████████████████████| 1.3 MB 63.5 MB/s 
     |████████████████████████████████| 120 kB 72.8 MB/s 
     |████████████████████████████████| 6.6 MB 61.1 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 115 kB 73.8 MB/s 
     |████████████████████████████████| 127 kB 62.9 MB/s 
  Created w

In [3]:
! pip install -U 'farm-haystack[pinecone]'==1.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 475 kB 28.4 MB/s 
     |████████████████████████████████| 981 kB 58.0 MB/s 
     |████████████████████████████████| 381 kB 69.5 MB/s 
     |████████████████████████████████| 50 kB 8.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 10.7 MB 49.0 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x3da6000 @  0x7f4c2bd5a615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 1.9 kB/s 
     |████████████████████████████████| 3.3 MB 48.4 MB/s 
     |████████████████████████

In [4]:
! pip show pinecone-client
! pip show transformers

Name: pinecone-client
Version: 2.0.10
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io/
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Proprietary License
Location: /usr/local/lib/python3.7/dist-packages
Requires: requests, typing-extensions, python-dateutil, pyyaml, loguru, dnspython, urllib3
Required-by: 
Name: transformers
Version: 4.13.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: tqdm, regex, numpy, importlib-metadata, requests, filelock, sacremoses, tokenizers, huggingface-hub, pack

In [5]:
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import tqdm
import random
from datasets import load_dataset
import pinecone
import opendatasets as od

# Model Selection

Unzip model file present in Drive

In [6]:
!unzip /content/drive/MyDrive/model.zip

Archive:  /content/drive/MyDrive/model.zip
   creating: notebooks/biencoder-arxiv/
  inflating: notebooks/biencoder-arxiv/pytorch_model.bin  
  inflating: notebooks/biencoder-arxiv/README.md  
  inflating: notebooks/biencoder-arxiv/vocab.txt  
  inflating: notebooks/biencoder-arxiv/tokenizer.json  
   creating: notebooks/biencoder-arxiv/1_Pooling/
  inflating: notebooks/biencoder-arxiv/1_Pooling/config.json  
  inflating: notebooks/biencoder-arxiv/config.json  
  inflating: notebooks/biencoder-arxiv/modules.json  
  inflating: notebooks/biencoder-arxiv/tokenizer_config.json  
  inflating: notebooks/biencoder-arxiv/config_sentence_transformers.json  
  inflating: notebooks/biencoder-arxiv/sentence_bert_config.json  
  inflating: notebooks/biencoder-arxiv/special_tokens_map.json  
   creating: notebooks/biencoder-arxiv/eval/


Specifying location for the old model and the trained model

In [7]:
DOCS = '/content/drive/MyDrive/Artemis.txt'
DATASET = '/content/arxiv/arxiv-metadata-oai-snapshot.json'
MODEL_PATH = '/content/notebooks/biencoder-arxiv'

# DOCS = '/notebooks/Artemis.txt'
# DOCS = '/notebooks/Artemis_launch.txt'
# MODEL_PATH = '/notebooks/biencoder-arxiv'

In [8]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

old_model_name = "msmarco-distilbert-base-tas-b"
old_model = SentenceTransformer(old_model_name)

# new_model_name = "/content/notebooks/biencoder-arxiv"
new_model_name = MODEL_PATH
new_model = SentenceTransformer(new_model_name)
# embeddings = model.encode(sentences)
# print(embeddings)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

We load the TAS-B model, a state-of-the-art model trained on MS MARCO

In [9]:
max_seq_length = 256
model_name = "msmarco-distilbert-base-tas-b"
# model_name = "icelab/spaceroberta"

org_model = SentenceTransformer(model_name)
org_model.max_seq_length = max_seq_length

# PineCone Operations

Logging into Pinecone and creating index negative-miness if not present

Uploading current documents to Pinecone in batch with a namespace to perform faster search

Function to query top 3 similar embeddings from Pinecone, we take the query and get the embeddings based on the model provided. For all the matches, we get the ID and the TEXT from the metadata. This is then convert to a result dictionary to be used for Haystack using the document store.

PineCone Wrapper to perform operations such as upload, query and delete for our needs.

In [10]:
class PineconeWrapper():
    def __init__(self, model, top_k=3, dimensions=768):
        pinecone.init(api_key='b368dee5-cb6b-4ddf-8858-abcc30c4acb7', environment='us-west1-gcp')
        # create new mining index if does not exist
        if 'negative-miness' not in pinecone.list_indexes():
            pinecone.create_index(
                'negative-miness', dimension=dimensions,
                metric='dotproduct', pods=1, pod_type='p1'  # limit of pods=1 for free plan (more pods == faster mining)
            )
        # connect
        self.index = pinecone.Index('negative-miness')
        print("Index Stats: ", self.index)

        self.batch_size = 16
        self.dimension = dimensions
        self.model = model
        self.top_k = top_k


    def upload_pinecone(self, haystack_docs, namespace):
        docs = [haystack_docs[i]['content'] for i in range(len(haystack_docs))]
        print("\nUPLOAD PINECONE START!*******!")
        docs_emb = self.model.encode(docs, convert_to_tensor=True, show_progress_bar=True)
        print(f"Document Embeddings Shape : {docs_emb.shape}")

        index_data = self.index.describe_index_stats()
        print(f"Index Data before Adding : {index_data}")
        totalVectorCount = int(index_data['totalVectorCount'])

        for i in tqdm.tqdm(range(0, len(docs_emb), self.batch_size)):
            i_end = min(i+self.batch_size, len(docs_emb))
            batch_emb = docs_emb[i:i_end, :].tolist()
            batch_data = docs[i:i_end]
            batch_metadata = [{"text": batch_data[i]} for i in range(0, len(batch_data))]
            batch_ids = [str(x+totalVectorCount) for x in range(i, i_end)]
            # print(f"Batch ID : {batch_ids}, Batch MetaData : {batch_metadata}")
            # print(f"Batch ID : {batch_ids}, Batch Embeddings : {batch_emb}")
            # print(f"Batch ID : {batch_ids}, Batch Data : {batch_data}")
            # upload to index
            upload_vectors = list(zip(batch_ids, batch_emb, batch_metadata))
            # print(f"Batch Upload Vectors : {upload_vectors}")
            self.index.upsert(vectors=upload_vectors, namespace=namespace)
        
        index_data = self.index.describe_index_stats()
        print(f"Index Data after Adding : {index_data}")
        print("\nUPLOAD PINECONE END!*******!")

    def delete_vectors(self, namespace=None):
        if not namespace:
            index_data = self.index.describe_index_stats()
            print(f"Index Data before deleting : {index_data}")

            to_delete = []
            for i in range(12263, 12294):
                to_delete.append(str(i))
            print(f"Index to Delete : {to_delete}")
            self.index.delete(ids = to_delete)

            index_data = self.index.describe_index_stats()
            print(f"Index Data after deleting : {index_data}")
        else:
            index_data = self.index.describe_index_stats()
            print(f"Index Data before deleting : {index_data}")

            self.index.delete(delete_all=True, namespace=namespace)

            index_data = self.index.describe_index_stats()
            print(f"Index Data after deleting : {index_data}")

    def query_pinecone(self, query, namespace):
        query_emb = self.model.encode(query).tolist()
        # print(query_emb.shape)
        # print(query_emb)
        
        # res = index.query([query_emb], top_k = 10)
        if namespace:
            res = self.index.query([query_emb], top_k = self.top_k, namespace=namespace, include_metadata=True)
        else:
            res = self.index.query([query_emb], top_k = self.top_k)
        # print(f"Pinecone Results : {res}")
        
        # ids = [match.id for match in res['results'][0]['matches']]
        # scores = [match.score for match in res['results'][0]['matches']]
        
        # print(ids)
        # print(scores)
        answers = []
        
        for match in res['results'][0]['matches']:
            vector_id = int(match.id)
            score = match.score
            # text = corpus[vector_id]
            try:
                text = match['metadata']['text']
            except:
                text = abstracts[vector_id]
                
            result_dict = {
                "content" : text,
                "context-type" : "text",
                "meta":{
                    "id" : vector_id,
                    "score": score
                }
            }
            answers.append(result_dict)
            # print(f"Score : {score}, ID : {vector_id}, TEXT : {text}")
        
        return answers

# LXML

In [ ]:
! pip uninstall lxml -y
! pip install libxml2
! pip install lxml==4.9.0

ERROR: Could not find a version that satisfies the requirement libxml2 (from versions: none)
ERROR: No matching distribution found for libxml2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.1 MB/s eta 0:00:0000:0100:01


# Integrating LFQA and Haystack

# Get Answers from LFQA

In [11]:
import torch
from haystack import Document
from haystack.generator.transformers import Seq2SeqGenerator
from haystack.utils import print_answers
# from haystack.utils.preprocessing import convert_files_to_docs
# from haystack.document_stores import BaseDocumentStore

INFO:haystack.modeling.model.optimization:apex not found, won't use it. See https://nvidia.github.io/apex/


Custom function to retrieve documents from input file

In [12]:
from pathlib import Path

def get_doc(doc_dir):
    text = Path(doc_dir).read_text().replace("\n", " ")
    text = text.replace('"', "")
    out = []
    threshold = 300
    for chunk in text.split('. '):
        if out and len(chunk)+len(out[-1]) < threshold:
            out[-1] += ' '+chunk+'.'
        else:
            out.append(chunk+'.')

    doc_results = []
    for doc in out:
        result_dict = {
                "content" : doc,
                "context-type" : "text",
                "meta":{
                    "id" : None,
                    "score": 0
                }
            }
        doc_results.append(result_dict)

    return doc_results

In [13]:
def get_answers_wrapper(query, doc_dir, model, namespace, top_k):
    doc_results = get_doc(doc_dir)
    # print(f"DOC RESULTS : {doc_results}")

    pineconeWrapper = PineconeWrapper(model, top_k=top_k, dimensions = 768)

    pineconeWrapper.upload_pinecone(doc_results, namespace)

    biencoder_results = pineconeWrapper.query_pinecone(query, namespace)
    document_store = []

    data = {}
    data['query'] = query
    data['document'] = doc_dir
    
    for idx, doc in enumerate(biencoder_results):
        i = str(idx)
        data["top_"+i+"_content"] = doc['content']
        data["top_"+i+"_dotscore"] = doc['meta']['score']
        document_store.append(Document(doc['content']))

    print(f"All Documents : {document_store}")

    generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

    result = generator.predict(
        query=query,
        documents = document_store,
        top_k=1
    )

    print_answers(result, details="minimum")
    
    answers = result['answers']
    for answer in answers:
        final_ans = answer.answer
        ans_score = answer.score
        break
    
    data['answer'] = final_ans
    data['score'] = ans_score
    # return final_ans
    return data

In [14]:
def run_inference_refactored(query, doc_dir, model, top_k=5):
    pineconeWrapper = PineconeWrapper(model, top_k=top_k)
    pineconeWrapper.delete_vectors(namespace='artemis')
    data = get_answers_wrapper(query, doc_dir, model, namespace='artemis', top_k=top_k)
    return data

Specify query and document dicrectory to use the file. Then we get the list of texts from the document. Now we pass the model and the query to top 3 documents from the query_pinecone function. Combined result contains the document from the texts as well as top 3 documents from query_pinecone function. For all the documents in combined results we create a document store. Then we pass this document for the LFQA to obtain the answers.

# Inference - Testing

In [ ]:
# query = "What is Artemis program named after?"
# query = "When will Artemis II be launched?"
# query = "When will Artemis III be launched?"
# query = "Describe Space Launch System rocket"
# query = "What are the goals of the Artemis program?"
# doc_dir = "/notebooks/Artemis.txt"
# query = "What were the main learnings from the Apollo program?"
# query = "What is the biggest trial for Orion?"
# query = "Why does Artemis carry mannequins?"
# query = "What are the objectives for the inaugural Artemis flight?"
# query = "Why is the Artemis team is willing to take more risks?"

# DOCS = "/notebooks/NASA_SOFIA.txt"
# query = "Why was Artemis I launch called off?"

# DOCS = "/notebooks/NASA_SOFIA.txt"
DOCS = "/content/NASA_SOFIA.txt"
# query = "Is there water in the moon?"
# query = "What is the evidence for the presence of water in the moon?"
query = "What does SOFIA detect in Clavius Crater?"
# query = "What guaranteed the presence of water in the moon?"
# query = "What can be the possible causes for water on the moon?"
# doc_dir = "/content/drive/MyDrive/Artemis.txt"
doc_dir = DOCS

In [ ]:
answers_new = run_inference_refactored(query, doc_dir, new_model, top_k=3)
answers_new['model_type'] = 'trained'

Index Stats:  <pinecone.index.Index object at 0x7fe4b850b6d0>
Index Data before deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 34}},
 'totalVectorCount': 12297.0}
Index Data after deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}
Index Stats:  <pinecone.index.Index object at 0x7fe4ba442850>

UPLOAD PINECONE START!*******!


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Document Embeddings Shape : torch.Size([34, 768])
Index Data before Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}


100%|██████████| 3/3 [00:00<00:00,  7.74it/s]
INFO:haystack.modeling.utils:Using devices: CUDA
INFO:haystack.modeling.utils:Number of GPUs: 1


Index Data after Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 34}},
 'totalVectorCount': 12297.0}

UPLOAD PINECONE END!*******!
All Documents : [<Document: {'content': 'This discovery indicates that water may be distributed across the lunar surface, and not limited to cold, shadowed places.  SOFIA has detected water molecules (H2O) in Clavius Crater, one of the largest craters visible from Earth, located in the Moon’s southern hemisphere.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': 'e3680e901655cc62f8af8e19eebd503a'}>, <Document: {'content': 'Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SOFIA was able to pick up the specific wavelength unique to water molecules, at 6.1 microns, and discovered a relatively surprising concentration in sunny Clavius Crater.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '2071c0cd417e4

In [ ]:
answers_old = run_inference_refactored(query, doc_dir, old_model, top_k=3)
answers_old['model_type'] = 'pretrained'

Index Stats:  <pinecone.index.Index object at 0x7fe4b17f4350>
Index Data before deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 34}},
 'totalVectorCount': 12297.0}
Index Data after deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}
Index Stats:  <pinecone.index.Index object at 0x7fe4b17f4d10>

UPLOAD PINECONE START!*******!


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Document Embeddings Shape : torch.Size([34, 768])
Index Data before Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}


100%|██████████| 3/3 [00:00<00:00,  7.85it/s]
INFO:haystack.modeling.utils:Using devices: CUDA
INFO:haystack.modeling.utils:Number of GPUs: 1


Index Data after Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 34}},
 'totalVectorCount': 12297.0}

UPLOAD PINECONE END!*******!
All Documents : [<Document: {'content': 'This discovery indicates that water may be distributed across the lunar surface, and not limited to cold, shadowed places.  SOFIA has detected water molecules (H2O) in Clavius Crater, one of the largest craters visible from Earth, located in the Moon’s southern hemisphere.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': 'e3680e901655cc62f8af8e19eebd503a'}>, <Document: {'content': 'Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SOFIA was able to pick up the specific wavelength unique to water molecules, at 6.1 microns, and discovered a relatively surprising concentration in sunny Clavius Crater.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '2071c0cd417e4

In [ ]:
print(f"Query : {query}\n")
print(f"Answer from trained model : {answers_new}\n")
print(f"Answer from base model : {answers_old}")

Query : What does SOFIA detect in Clavius Crater?

Answer from trained model : {'query': 'What does SOFIA detect in Clavius Crater?', 'top_0_content': 'This discovery indicates that water may be distributed across the lunar surface, and not limited to cold, shadowed places.  SOFIA has detected water molecules (H2O) in Clavius Crater, one of the largest craters visible from Earth, located in the Moon’s southern hemisphere.', 'top_0_dotscore': 114.901901, 'top_1_content': 'Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SOFIA was able to pick up the specific wavelength unique to water molecules, at 6.1 microns, and discovered a relatively surprising concentration in sunny Clavius Crater.', 'top_1_dotscore': 112.091789, 'top_2_content': 'Whether the water SOFIA found is easily accessible for use as a resource remains to be determined.', 'top_2_dotscore': 103.595711, 'answer': "SOFIA has detected water molecules (H2O) in Clavius Crater, one of the largest craters 

In [ ]:
import pandas as pd

df = pd.DataFrame([answers_new, answers_old])
# print(df)
df

,query,top_0_content,top_0_dotscore,top_1_content,top_1_dotscore,top_2_content,top_2_dotscore,answer,score,model_type
0,What does SOFIA detect in Clavius Crater?,This discovery indicates that water may be distributed across the lunar surf...,114.901901,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",112.091789,Whether the water SOFIA found is easily accessible for use as a resource rem...,103.595711,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,trained
1,What does SOFIA detect in Clavius Crater?,This discovery indicates that water may be distributed across the lunar surf...,109.044266,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",107.474091,NASA’s SOFIA Discovers Water on Sunlit Surface of Moon NASA’s Stratospheric...,97.208694,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,pretrained


# CSV Export

Function to return for a list of query_mapping values.

In [15]:
import pandas as pd

def get_csv(query_mapping):
    all_data = []
    for k,v in query_mapping.items():
        query = k
        doc_dir = v
        answers_new = run_inference_refactored(query, doc_dir, new_model, top_k=3)
        answers_new['model_type'] = 'trained'
        all_data.append(answers_new)

        answers_old = run_inference_refactored(query, doc_dir, old_model, top_k=3)
        answers_old['model_type'] = 'pretrained'
        all_data.append(answers_old)
    
    df = pd.DataFrame(all_data)
    return df

In [19]:
# query_mapping = {
#     "Goals of the Artemis program?" : "/content/Artemis.txt",
#     "What does SOFIA detect in Clavius Crater?" : "/content/NASA_SOFIA.txt",
#     "Why was artemis launch delayed?" : "/content/Artemis_launch.txt",
#     "What is the key objective of Perserverance mission?" : "/content/NASA_PerserveranceRover.txt"
# }
query_mapping = {
    "Goals of the Artemis program?" : "/content/Artemis.txt",
    "What does SOFIA detect in Clavius Crater?" : "/content/NASA_SOFIA.txt",
    "Why was artemis launch delayed?" : "/content/Artemis_launch.txt",
    "What is the key objective of Perserverance mission?" : "/content/NASA_PerserveranceRover.txt",

    "When will Artemis III be launched?" : "/content/Artemis.txt",
    "What were the main learnings from the Apollo program?" : "/content/Artemis.txt",
    "What is the biggest trial for Orion?" : "/content/Artemis.txt",
    "Why does Artemis carry mannequins?" : "/content/Artemis.txt",
    "When will Artemis II be launched?" : "/content/Artemis.txt",

    "What were the possible solutions considered by NASA after the launch was delayed?" : "/content/Artemis_launch.txt",
    "Why did NASA face criticism?" : "/content/Artemis_launch.txt",
    "What were the possible reasons for Hydrogen leak?" : "/content/Artemis_launch.txt",
    "What are the next steps taken after the launch was called off?" : "/content/Artemis_launch.txt",

    "Mention some possible reasons for the creation of water in the moon?" : "/content/NASA_SOFIA.txt",
    "What guaranteed the presence of water in the moon?" : "/content/NASA_SOFIA.txt",
    "How did SOFIA inspire future missions?" : "/content/NASA_SOFIA.txt",
    "What questions did the discovery of water in the moon led to?" : "/content/NASA_SOFIA.txt",
    "How is the water stored in the moon?" : "/content/NASA_SOFIA.txt",

    "What are the next steps for the Perserverance mission?" : "/content/NASA_PerserveranceRover.txt",

    "Who are the competitors for NASA?" : "/content/Artemis_launch.txt",
    "What inspired NASA to name the mission as Artemis?" : "/content/Artemis_launch.txt",

    "Why was the Artemis program established?" : "/content/Artemis.txt",
    "Briefly describe Orion spacecraft" : "/content/Artemis.txt",
    "What are some futures plan after the Artemis launch?" : "/content/Artemis.txt",
    "How powerful is the sls rocket?" : "/content/Artemis.txt",
    "Why is orion not planned to carry crews in the initial mission?" : "/content/Artemis.txt",


}

df = get_csv(query_mapping)
df

Index Stats:  <pinecone.index.Index object at 0x7f80c242e550>
Index Data before deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}
Index Data after deleting : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}
Index Stats:  <pinecone.index.Index object at 0x7f80c15e13d0>

UPLOAD PINECONE START!*******!


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document Embeddings Shape : torch.Size([31, 768])
Index Data before Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}


100%|██████████| 2/2 [00:00<00:00,  3.07it/s]
INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt out at anytime by setting HAYSTACK_TELEMETRY_ENABLED="False" as an environment variable or by calling disable_telemetry(). More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CUDA
INFO:haystack.modeling.utils:Number of GPUs: 1


Index Data after Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 31}},
 'totalVectorCount': 12294.0}

UPLOAD PINECONE END!*******!
All Documents : [<Document: {'content': 'Goals of the Artemis program include landing diverse crews of astronauts on the moon and exploring the shadowy lunar south pole for the first time.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '1a0baedce47126dd858f7fe8c6846bd5'}>, <Document: {'content': 'The ultimate test All of the objectives for the inaugural Artemis flight will demonstrate capabilities necessary for when Orion carries humans to deep space.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': 'd6187e57914c7047837dc9f0ebec8483'}>, <Document: {'content': " It's no coincidence that the Artemis program is named for the twin sister of Apollo from Greek mythology. Artemis will pick up where the famed Apollo program left o

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document Embeddings Shape : torch.Size([31, 768])
Index Data before Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}},
 'totalVectorCount': 12263.0}


100%|██████████| 2/2 [00:00<00:00,  4.47it/s]
INFO:haystack.modeling.utils:Using devices: CUDA
INFO:haystack.modeling.utils:Number of GPUs: 1


Index Data after Adding : {'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 12263}, 'artemis': {'vector_count': 31}},
 'totalVectorCount': 12294.0}

UPLOAD PINECONE END!*******!
All Documents : [<Document: {'content': " Orion won't carry a crew on this initial mission, but it will be full of data from the flight -- including sensors attached to some very necessary passengers.", 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '519535be1936cb42d49369bab955fc28'}>, <Document: {'content': 'Atop the rocket is the Orion spacecraft, designed to carry a crew through deep space and safely return the astronauts to Earth.', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '70a427ca1072e8f75d22bd0da386579b'}>, <Document: {'content': 'The ultimate test All of the objectives for the inaugural Artemis flight will demonstrate capabilities necessary for when Orion carries humans to deep space.', 'content_type': 'text',

,query,document,top_0_content,top_0_dotscore,top_1_content,top_1_dotscore,top_2_content,top_2_dotscore,answer,score,model_type
0,Goals of the Artemis program?,/content/Artemis.txt,Goals of the Artemis program include landing diverse crews of astronauts on ...,114.100327,The ultimate test All of the objectives for the inaugural Artemis flight wil...,107.317284,It's no coincidence that the Artemis program is named for the twin sister o...,106.329231,Artemis is the name for the twin sister of Apollo. Apollo was the first mann...,None,trained
1,Goals of the Artemis program?,/content/Artemis.txt,Goals of the Artemis program include landing diverse crews of astronauts on ...,108.801666,The ultimate test All of the objectives for the inaugural Artemis flight wil...,101.499428,It's no coincidence that the Artemis program is named for the twin sister o...,100.712280,Artemis is the name for the twin sister of Apollo. Apollo was the first mann...,None,pretrained
2,What does SOFIA detect in Clavius Crater?,/content/NASA_SOFIA.txt,This discovery indicates that water may be distributed across the lunar surf...,114.901901,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",112.091789,Whether the water SOFIA found is easily accessible for use as a resource rem...,103.595711,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,trained
3,What does SOFIA detect in Clavius Crater?,/content/NASA_SOFIA.txt,This discovery indicates that water may be distributed across the lunar surf...,109.044266,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",107.474091,NASA’s SOFIA Discovers Water on Sunlit Surface of Moon NASA’s Stratospheric...,97.208694,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,pretrained
4,Why was artemis launch delayed?,/content/Artemis_launch.txt,"The scrubbed launch is for Artemis I, a weekslong, uncrewed mission that wi...",105.484688,NASA Calls Off Artemis I Moon Launch For a Second Time: Here's Why For the ...,105.399925,"Because NASA has so much invested in this one rocket, a catastrophic failur...",102.224884,"Artemis was supposed to launch today, but it was delayed due to weather.",None,trained
5,Why was artemis launch delayed?,/content/Artemis_launch.txt,"The scrubbed launch is for Artemis I, a weekslong, uncrewed mission that wi...",99.890770,NASA Calls Off Artemis I Moon Launch For a Second Time: Here's Why For the ...,98.610710,Monday’s launch was called off when a faulty sensor reported that one of the...,96.284706,"Artemis was supposed to launch on Monday, but it was delayed due to the weat...",None,pretrained
6,What is the key objective of Perserverance mission?,/content/NASA_PerserveranceRover.txt,“What motivates us as engineers and scientists exploring another planet is t...,111.980064,"In a new video, Jessica Samuels – the Perseverance surface operations missi...",110.029221,The Mars 2020 Perseverance mission is part of NASA’s Moon to Mars explorati...,109.163208,The main objective of the mission is to look for signs of ancient microbial ...,None,trained
7,What is the key objective of Perserverance mission?,/content/NASA_PerserveranceRover.txt,“What motivates us as engineers and scientists exploring another planet is t...,102.011833,The Mars 2020 Perseverance mission is part of NASA’s Moon to Mars explorati...,99.707008,"In a new video, Jessica Samuels – the Perseverance surface operations missi...",99.603760,The main objective of the mission is to look for signs of ancient microbial ...,None,pretrained
8,When will Artemis III be launched?,/content/Artemis.txt,"When Artemis I launches on August 29, the uncrewed mission will test every n...",106.940842,"After launching from Earth, Artemis I will go on a 42-day mission. During th...",104.011032,Goals of the Artemis program include landing diverse crews of astronauts on ...,103.839500,Artemis II is scheduled to launch in 2021. Artemis II

In [20]:
df

,query,document,top_0_content,top_0_dotscore,top_1_content,top_1_dotscore,top_2_content,top_2_dotscore,answer,score,model_type
0,Goals of the Artemis program?,/content/Artemis.txt,Goals of the Artemis program include landing diverse crews of astronauts on ...,114.100327,The ultimate test All of the objectives for the inaugural Artemis flight wil...,107.317284,It's no coincidence that the Artemis program is named for the twin sister o...,106.329231,Artemis is the name for the twin sister of Apollo. Apollo was the first mann...,None,trained
1,Goals of the Artemis program?,/content/Artemis.txt,Goals of the Artemis program include landing diverse crews of astronauts on ...,108.801666,The ultimate test All of the objectives for the inaugural Artemis flight wil...,101.499428,It's no coincidence that the Artemis program is named for the twin sister o...,100.712280,Artemis is the name for the twin sister of Apollo. Apollo was the first mann...,None,pretrained
2,What does SOFIA detect in Clavius Crater?,/content/NASA_SOFIA.txt,This discovery indicates that water may be distributed across the lunar surf...,114.901901,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",112.091789,Whether the water SOFIA found is easily accessible for use as a resource rem...,103.595711,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,trained
3,What does SOFIA detect in Clavius Crater?,/content/NASA_SOFIA.txt,This discovery indicates that water may be distributed across the lunar surf...,109.044266,"Using its Faint Object infraRed CAmera for the SOFIA Telescope (FORCAST), SO...",107.474091,NASA’s SOFIA Discovers Water on Sunlit Surface of Moon NASA’s Stratospheric...,97.208694,"SOFIA has detected water molecules (H2O) in Clavius Crater, one of the large...",None,pretrained
4,Why was artemis launch delayed?,/content/Artemis_launch.txt,"The scrubbed launch is for Artemis I, a weekslong, uncrewed mission that wi...",105.484688,NASA Calls Off Artemis I Moon Launch For a Second Time: Here's Why For the ...,105.399925,"Because NASA has so much invested in this one rocket, a catastrophic failur...",102.224884,"Artemis was supposed to launch today, but it was delayed due to weather.",None,trained
5,Why was artemis launch delayed?,/content/Artemis_launch.txt,"The scrubbed launch is for Artemis I, a weekslong, uncrewed mission that wi...",99.890770,NASA Calls Off Artemis I Moon Launch For a Second Time: Here's Why For the ...,98.610710,Monday’s launch was called off when a faulty sensor reported that one of the...,96.284706,"Artemis was supposed to launch on Monday, but it was delayed due to the weat...",None,pretrained
6,What is the key objective of Perserverance mission?,/content/NASA_PerserveranceRover.txt,“What motivates us as engineers and scientists exploring another planet is t...,111.980064,"In a new video, Jessica Samuels – the Perseverance surface operations missi...",110.029221,The Mars 2020 Perseverance mission is part of NASA’s Moon to Mars explorati...,109.163208,The main objective of the mission is to look for signs of ancient microbial ...,None,trained
7,What is the key objective of Perserverance mission?,/content/NASA_PerserveranceRover.txt,“What motivates us as engineers and scientists exploring another planet is t...,102.011833,The Mars 2020 Perseverance mission is part of NASA’s Moon to Mars explorati...,99.707008,"In a new video, Jessica Samuels – the Perseverance surface operations missi...",99.603760,The main objective of the mission is to look for signs of ancient microbial ...,None,pretrained
8,When will Artemis III be launched?,/content/Artemis.txt,"When Artemis I launches on August 29, the uncrewed mission will test every n...",106.940842,"After launching from Earth, Artemis I will go on a 42-day mission. During th...",104.011032,Goals of the Artemis program include landing diverse crews of astronauts on ...,103.839500,Artemis II is scheduled to launch in 2021. Artemis II

In [21]:
df.to_csv("Results.csv", index=None)

In [22]:
df.to_excel("Results.xlsx", index=None)